<a href="https://colab.research.google.com/github/Defesa-MT/CEGERD/blob/main/MAPA_INMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
!pip install requests
!pip install folium

# Importações

In [1]:
import requests
import pandas as pd
import folium

#Dados

In [10]:
start_date = '2024-03-27'
end_date = '2024-03-28'
station_ids = [
    'A908', 'A924', 'A909', 'A934', 'A910', 'A927', 'A905', 'A912',
    'A926', 'A913', 'A919', 'A901', 'A930', 'A906', 'A932', 'A933',
    'A914', 'A920', 'A928', 'A929', 'A915', 'A937', 'A935', 'A923',
    'A916', 'A907', 'A944', 'A903', 'A936', 'A931', 'A921', 'A942',
    'A911', 'A943', 'A917', 'A904', 'A902', 'A922', '83270', '83361',
    '83309', '83319', '83364', '83363'
]

  # IDs das estações
token = "seu tolk
data_stations = []  # Lista para armazenar os dados de cada estação

for station_id in station_ids:
    url = f"https://apitempo.inmet.gov.br/token/estacao/diaria/{start_date}/{end_date}/{station_id}/{token}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        for record in data:
            record['CD_ESTACAO'] = station_id  # Adiciona o ID da estação a cada registro
            data_stations.append(record)
    except requests.RequestException as e:
        print(f"Erro ao obter dados para a estação {station_id}: {e}")

# Criando um DataFrame a partir dos dados coletados
df = pd.DataFrame(data_stations)

# Exibindo o DataFrame para visualização no Google Colab
print(df)

# Salvando o DataFrame em um arquivo CSV
filename = 'dados_estacoes.csv'
df.to_csv(filename, index=False)

# Caso você esteja no Google Colab, utilize o seguinte para baixar o arquivo CSV para sua máquina local
# from google.colab import files
# files.download(filename)


   UMID_MED  DT_MEDICAO                 DC_NOME UMID_MIN TEMP_MED CHUVA  \
0      92.3  2024-03-27                ÁGUA BOA       83     23.7    12   
1      None  2024-03-28                ÁGUA BOA     None     None  26.2   
2      None  2024-03-27           ALTA FLORESTA       93     25.3   4.2   
3      None  2024-03-28           ALTA FLORESTA     None     None  28.2   
4      None  2024-03-27           ALTO ARAGUAIA       76     21.7    30   
..      ...         ...                     ...      ...      ...   ...   
83       94  2024-03-28    NOVA XAV.(XAVANTINA)      NaN     None  73.8   
84       85  2024-03-27  PADRE RICARDO REMETTER      NaN     None  10.6   
85       91  2024-03-28  PADRE RICARDO REMETTER      NaN     None  13.4   
86     None  2024-03-27             SÃO VICENTE      NaN     None  None   
87     None  2024-03-28             SÃO VICENTE      NaN     None  None   

     VL_LATITUDE TEMP_MIN TEMP_MAX  UF VEL_VENTO_MED CD_ESTACAO  VL_LONGITUDE  \
0   -14.01638888  

# MAPA

In [11]:
# Criando o mapa centrado nas coordenadas médias
mapa = folium.Map(location=[df['VL_LATITUDE'].astype(float).mean(), df['VL_LONGITUDE'].astype(float).mean()], zoom_start=5)

# Adicionando marcadores para cada estação com a cor baseada na ocorrência de chuva
for _, estacao in df.iterrows():
    # Checando se o valor de 'CHUVA' é um número e atribuindo a cor baseado no volume de chuva
    if pd.notnull(estacao['CHUVA']):
        chuva = float(estacao['CHUVA'])
        if chuva > 60:
            # Cor vermelha para chuva acima de 60mm
            icone = folium.Icon(icon="cloud", prefix="fa", color="red")
        elif 30 < chuva <= 60:
            # Cor laranja para chuva entre 30 e 60mm
            icone = folium.Icon(icon="cloud", prefix="fa", color="orange")
        elif 20 < chuva <= 30:
            # Cor amarela para chuva entre 20 e 30mm
            icone = folium.Icon(icon="cloud", prefix="fa", color="yellow")
        elif chuva <= 20:
            # Cor verde para chuva até 20mm
            icone = folium.Icon(icon="cloud", prefix="fa", color="green")
    else:
        # Cor cinza para estação sem dados de chuva
        icone = folium.Icon(color="lightgray")

    folium.Marker(
        location=[float(estacao['VL_LATITUDE']), float(estacao['VL_LONGITUDE'])],
        popup=f"{estacao['DC_NOME']} - Chuva: {estacao['CHUVA']}mm",
        icon=icone
    ).add_to(mapa)

# Adicionando uma legenda customizada
legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 200px; height: 130px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     ">&nbsp; Legenda da Chuva <br>
      &nbsp; <i class="fa fa-cloud" style="color:green"></i>&nbsp; Até 20mm <br>
      &nbsp; <i class="fa fa-cloud" style="color:yellow"></i>&nbsp; De 20 a 30mm <br>
      &nbsp; <i class="fa fa-cloud" style="color:orange"></i>&nbsp; De 30 a 60mm <br>
      &nbsp; <i class="fa fa-cloud" style="color:red"></i>&nbsp; Acima de 60mm <br>
      &nbsp; <i class="fa fa-map-marker" style="color:lightgray"></i>&nbsp; Sem chuva / Sem dados
     </div>
     '''
mapa.get_root().html.add_child(folium.Element(legend_html))

# Salvando o mapa em um arquivo HTML
mapa.save('mapa_estacoes_chuva_atualizado.html')

# O mapa agora pode ser aberto em um navegador web.

# Mostrando o mapa
mapa


<ipython-input-11-8e6792d2c0a5>:17: UserWarning: color argument of Icon should be one of: {'green', 'darkpurple', 'black', 'purple', 'lightblue', 'beige', 'lightred', 'pink', 'darkred', 'darkblue', 'lightgray', 'white', 'red', 'cadetblue', 'lightgreen', 'gray', 'darkgreen', 'blue', 'orange'}.
  icone = folium.Icon(icon="cloud", prefix="fa", color="yellow")
